<a href="https://colab.research.google.com/github/ewilwertuoz/Transfer-Learning/blob/main/Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning: дообучение сети.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import os
import copy
import time
from PIL import Image
from tqdm import tqdm_notebook

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
from torchvision import datasets, models, transforms

## Загрузка предобученной сети

Загружаем предобученный resnet из репозитория pytorch:

In [ ]:
# модель resnet-18
model = models.resnet18(pretrained=True)
model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
resnet_transforms = transforms.Compose([
        transforms.Resize(256), # размер каждой картинки будет приведен к 256*256
        transforms.CenterCrop(224), # у картинки будет вырезан центральный кусок размера 224*224
        transforms.ToTensor(), # картинка из питоновского массива переводится в формат torch.Tensor
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # значения пикселей картинки нормализуются
    ])

In [ ]:
train_data = datasets.ImageFolder('', transform=resnet_transforms)
val_data = datasets.ImageFolder('', transform=resnet_transforms)
test_data = datasets.ImageFolder('', transform=resnet_transforms)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=64, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)

### Заморозка весов и замена последнего слоя

In [ ]:
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(512, 70)

Заморозим все сверточные слои:

In [ ]:
for i, layer in enumerate(model.children()):
  if i < 9:
    for param in layer.parameters():
      param.requires_grad = False

Перенесем нашу нейросеть на GPU, если GPU доступен:

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
device

In [ ]:
model = model.to(device)

### Обучение сети

In [ ]:
# выбираем функцию потерь
loss_fn = torch.nn.CrossEntropyLoss()

# выбираем алгоритм оптимизации и learning_rate
learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

Импортируем нужные модули и пишем нужные команды для Tensorboard:

In [ ]:
import os

if not os.path.exists('logs'):
    os.mkdir('logs')

%load_ext tensorboard

In [ ]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter("logs")

In [ ]:
%tensorboard --logdir=./logs

Функция обучения сети:

In [ ]:
def evaluate(model, dataloader, loss_fn):

    losses = []

    num_correct = 0
    num_elements = len(dataloader)

    for i, batch in enumerate(dataloader):

        # так получаем текущий батч
        X_batch, y_batch = batch

        with torch.no_grad():
            X_batch = X_batch.to(device)
            logits = model(X_batch)

            y_batch = y_batch.to(device)
            loss = loss_fn(logits, y_batch)
            losses.append(loss.item())

            y_pred = torch.argmax(logits, dim=1)

            num_correct += torch.sum(y_pred == y_batch)

    accuracy = num_correct / num_elements

    return accuracy, np.mean(losses)

def train(model, loss_fn, optimizer, n_epoch=3):

    num_iter = 0

    # цикл обучения сети
    for epoch in range(n_epoch):

        print("Epoch:", epoch)

        model.train(True)

        for i, batch in enumerate(train_loader):
            # так получаем текущий батч
            X_batch, y_batch = batch

            # forward pass (получение ответов на батч картинок)
            X_batch = X_batch.to(device)
            logits = model(X_batch)

            # вычисление лосса от выданных сетью ответов и правильных ответов на батч
            y_batch = y_batch.to(device)
            loss = loss_fn(logits, y_batch)


            loss.backward() # backpropagation (вычисление градиентов)
            optimizer.step() # обновление весов сети
            optimizer.zero_grad() # обнуляем веса

            #########################
            # Логирование результатов
            num_iter += 1
            writer.add_scalar('Loss/train', loss.item(), num_iter)

            # вычислим accuracy на текущем train батче
            model_answers = torch.argmax(logits, dim=1)
            train_accuracy = torch.sum(y_batch == model_answers) / len(y_batch)
            writer.add_scalar('Accuracy/train', train_accuracy, num_iter)
            #########################

        # после каждой эпохи получаем метрику качества на валидационной выборке
        model.train(False)

        val_accuracy, val_loss = evaluate(model, val_loader, loss_fn=loss_fn)

        writer.add_scalar('Loss/val', val_loss.item(), num_iter)
        writer.add_scalar('Accuracy/val', val_accuracy, num_iter)


    return model

Обучаем сеть:

In [ ]:
model = train(model, loss_fn, optimizer, n_epoch=3)

Проверим метрику accuracy на тренировочной и тестовой выборках:

In [ ]:
train_accuracy, _ = evaluate(model, train_loader);
print('Train accuracy is', train_accuracy)

In [ ]:
test_accuracy, _ = evaluate(model, test_loader);
print('Test accuracy is', test_accuracy)

Загрузка графиков обучения на Tensorboard dev:

In [ ]:
# !tensorboard dev upload --logdir=./logs \
# --name "My latest experiment" \
# --description "Simple comparison of several hyperparameters"

## Сохранение модели

Сохранение модели:

In [ ]:
torch.save(model, 'model.pt')

Загрузка модели из сохраненного чекпоинта:

In [ ]:
model_new = torch.load('model.pt')

Проверим, что модель загрузилась и что она точно та же, что мы тестировали выше:

Загруженную модель можно, например, дообучить еще несколько эпох:

In [ ]:
model_new, train_losses, val_losses, val_accuracies = train(model_new, n_epoch=5)